In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
import pandas as pd
#from datetime import datetime
#import pandas.io.sql as psql

####################### Opening Connection with Database ####################

conn = psycopg2.connect(database="postgres", user = "postgres", password = "pass123", host = "127.0.0.1", port = "5432")
cursor = conn.cursor()
# Reaad from sql into a table and then into a DataFrame
my_table    = pd.read_sql('select * from day_wise_offer_statistics ', conn)
my_df= pd.DataFrame(data=my_table)


In [3]:
####################### Calculation and Data Processing starts here #################

#Calculate the number of days an item is available on platform
my_df['day_available'] = my_df.groupby('oid').cumcount() + 1

# For a given Offer sort them in asc order of days available
my_df.sort_values(ascending = True, by = ['oid','day_available'],inplace=True)
my_df=my_df.reset_index().drop(columns='index',axis=1)

# Calculate the weighted score based on various factors
my_df['wt_raw_score']=0.22*my_df['chat_count']+0.15*my_df['comment_count']+0.37*my_df['deal_count']+ 0.22*my_df['favourite_count']+0.11*my_df['view_count']

# Calculate the day-on-day increase in the weighted score 
my_df['dod_increase']=(my_df['wt_raw_score']-my_df['wt_raw_score'].shift(1))/(my_df['wt_raw_score'].shift(1))

# Day-on-day increase metric should be reset to zero for the 1st day of every offer id
for i in range(0,len(my_df)):
    if (my_df['day_available'][i]==1) :
        my_df['dod_increase'][i]=0
    else:
        pass

# Taking the cumulative sum of the day-of-day increase by OFFER ID
my_df['cumlt_dod_score'] = my_df.groupby(['oid'])['dod_increase'].apply(lambda x: x.cumsum())

# Creating a table containing the Cumulative score for all the unique 'Available/Not sold' offer as on the n-th day 
top_scores=(my_df[(my_df['day_available'] == 7) & (my_df['available']==1)][['oid', 'vertical_id','item_type','identity_id','stat_date','day_available', 'cumlt_dod_score']].reset_index().drop(columns = 'index'))

# Sorting that table to get the top cumulative scores for TRENDING OFFERS
top_scores= top_scores.sort_values(ascending=False,by='cumlt_dod_score')

# Taking the cumulative sum of the day-of-day increase by CATEGORY 
top_category_scores= top_scores.groupby(['vertical_id'])['cumlt_dod_score'].sum()

# Declaring a dataframe to convert the series data into a df for easier data handling
top_categories = pd.DataFrame(columns=['vertical_Id','top_category_score'])
top_categories['vertical_Id']= top_category_scores.index
top_categories['top_category_score']=top_category_scores.values

# Sorting that table to get the top cumulative scores for POPULAR CATEGORIES
top_categories = top_categories.sort_values(ascending=False,by='top_category_score')



<ipython-input-3-b085fcd39067>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  my_df['dod_increase'][i]=0


In [4]:
####################### Output of Top 5 Trending Offers and Popular Categories #########################

top_scores= top_scores.head(5)
top_categories= top_categories.head(5)

print (top_scores)
print (top_categories)

######## To write back to an excel sheet ######       
#my_df.to_csv('C:/Users/admin/Desktop/Work/Trending Popular/from_postgresql.csv')
#print (my_df[['oid','day_available','available','wt_raw_score','dod_increase','cumlt_dod_score']].head(40))
# short_df= my_df[['oid','vertical_id','day_available','available','wt_raw_score','dod_increase','cumlt_dod_score']]
# short_df.to_csv('C:/Users/admin/Desktop/Work/Trending Popular/calculated_cols.csv')


   oid     vertical_id item_type identity_id   stat_date  day_available  \
5    4     Electronics   Product       User1  2021-05-18              7   
0    1     Electronics   Product       User1  2021-05-18              7   
7    6          Sports   Product       User5  2021-05-18              7   
10   9     PestControl   Service       User4  2021-05-18              7   
9    8  HomeAppliances   Product       User6  2021-05-18              7   

    cumlt_dod_score  
5         11.091597  
0          8.601500  
7          7.517018  
10         3.277699  
9          3.269712  
      vertical_Id  top_category_score
3     Electronics           19.693097
9          Sports            7.517018
7     PestControl            3.277699
6  HomeAppliances            3.269712
0           Books            3.239195


In [5]:
####################### Opening SQL connection to write back the results of TRENDING OFFER to the database #################

connection = psycopg2.connect(database="postgres", user = "postgres", password = "pass123", host = "127.0.0.1", port = "5432")
cursor = connection.cursor()
cols = ",".join([str(i) for i in top_scores.columns.tolist()])
print (cols)
####### Comment out this portion once the table update is complete #####

# for i,row in top_scores.iterrows():
#     sql = "INSERT INTO trending_offer (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
#     cursor.execute(sql, tuple(row))
#connection.commit()

oid,vertical_id,item_type,identity_id,stat_date,day_available,cumlt_dod_score


In [6]:
####################### Opening SQL connection to write back the results of POPULAR CATEGORIES to the database #################

connection = psycopg2.connect(database="postgres", user = "postgres", password = "pass123", host = "127.0.0.1", port = "5432")
cursor = connection.cursor()
cols = ",".join([str(i) for i in top_categories.columns.tolist()])
print (cols)
####### Comment out this portion once the table update is complete #####
# for i,row in top_categories.iterrows():
#     sql = "INSERT INTO popular_categories (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
#     cursor.execute(sql, tuple(row))
connection.commit()

vertical_Id,top_category_score
